# Exec Summary

This notebook goes into a lot of detail and presents all the math to justify using a Bayesian approach, but if you want to skip the math and just get the benefits, skip to the end and use the Python functions test_non_inferiority and select_best_variant which should be sufficient to support our CX analysis with 3 variants and a control.


## Problem

When launching new web or mobile features, engineering teams face a common dilemma:

- **Limited traffic allocation**: New features get only 5-10% of traffic to minimize risk
- **Multiple variants**: Design teams often propose 3-5 different implementations
- **Small sample sizes**: At launch because of controlled releases, each variant may only see hundreds or low thousands of users
- **Need for speed**: We need fast decisions on which variants are best to iterate or scale

**Traditional NHST fails here**: With small, unbalanced samples (e.g., 7,000 control vs. 150 per variant), statistical tests either:
- Fail to reach significance (underpowered, β > 80%)
- Require weeks of data collection
- Is unwieldy to compare more than 2 variants at a time

## The Bayesian Solution

Bayesian methods excel precisely where NHST struggles:

### 1. **Works with Small Samples**
- Incorporates prior knowledge (historical conversion rates)
- Updates beliefs incrementally as data arrives
- Provides meaningful conclusions even with n=150 per variant
- No arbitrary "minimum sample size" requirement

### 2. **Handles Unbalanced Allocation Naturally**
- 90% control, 10% variants? No problem.
- Each variant can have different sample sizes
- No need for equal allocation or "balanced designs"
- Protects existing user experience while testing

### 3. **Scales to Many Variants Effortlessly**
- Compare 3, 5, 10, or 100 variants simultaneously
- Single coherent analysis—no multiple comparison penalties
- Direct answer: P(A is best) = 31%, P(B is best) = 47%, P(C is best) = 22%

### 4. **Provides Actionable Probabilities**
- NHST says: "Cannot reject H₀" (not actionable)
- Bayesian says: "47% chance B is best, 22% it's worse than control" (actionable)
- Direct business decision: Deploy B with quantified risk

### 5. **Allows Continuous Monitoring**
- Can check results anytime without "p-hacking" concerns
- Update posteriors as new data arrives
- Stop early if clear winner emerges
- Continue if more certainty needed—mathematically rigorous


## Key Benefits Summary

| Aspect | Traditional NHST | Bayesian Approach |
|--------|-----------------|-------------------|
| Small samples | Underpowered, inconclusive | Works well with prior knowledge |
| Unbalanced allocation | Loses efficiency | No problem |
| Multiple variants | Complex corrections needed | Natural single analysis |
| Interpretation | p-value (hard to explain) | Probability (intuitive) |
| Decision making | Binary reject/fail | Quantified risk/confidence |
| Continuous monitoring | Forbidden (p-hacking) | Allowed and rigorous |
| Time to decision | Weeks (need larger n) | Days (works with small n) |

## Bottom Line

For modern product development with:
- **Rapid iteration cycles**
- **Risk-averse traffic allocation**
- **Multiple design options**
- **Small initial samples**

**Bayesian methods provide:**
- Faster decisions (days vs. weeks)
- Better use of limited data
- Clear, business-friendly outputs
- Quantified confidence for risk management

This enables product teams to launch confidently, iterate quickly, and scale successful features—all while protecting the existing user experience.

## Implementation

The two standalone Python functions in this notebook provide:
1. `test_non_inferiority()` - Verify new features don't degrade experience
2. `select_best_variant()` - Choose winning variant with probability

Both work with any sample sizes and scale to any number of variants.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm
from statsmodels.stats.proportion import confint_proportions_2indep
from statsmodels.stats.proportion import proportions_ztest
# import the beta function from scipy.special
from scipy.special import beta as beta_function
from scipy.stats import beta as beta_dist


# In Depth Analysis of the 2 methodologies

When evaluating new user experiences (UX) — such as launching **passkeys** and measuring their impact on abandonment rates — our CX team has provided us with 3 variants $A_1$, $A_2$ and $A_3$ for the passkey creation experience. After launch we need to make sure that adding passkey creation does not degrade the success rate of our current CX significantly by comparing each variant to a control group that will remain on our legacy pages, then we have to decide which of the variants performs best.
  
This notebook compares two major approaches:

- **Null Hypothesis Significance Testing (NHST)** — the long-standing statistical framework, widely used but often conceptually tricky and difficult to interpret in practical decision-making.
- **Bayesian methods** — increasingly popular because they offer more flexibility, better ability to work on small samples and produce results that are often easier to interpret directly when deciding on actions (e.g., when to shift more traffic from a control to a new variant).

---

## Test Setup: Control Group vs. Variants

For the sake of the discussion let's assume an existing digital identity creation flow with a **completion rate of ~20%** (meaning ~80% of users abandon).  

Our test design:

- Keep **x%** of traffic on the current experience as the **control group** C.
- Send the remaining traffic to one or more **variants** $A_1$, $A_2$, $A_3$.

The test will be conducted in 2 steps. First we need to determine that each new experience is **no worse** than the current one, accepting small degradation as unavoidable since we are adding more pages and clicks — then after establishing we haven't degraded the experience, shifting more traffic to the variants and deciding which is the better-performing variant.

The type of A/B test — where the first goal is to ensure a new design does **not degrade** the experience — is called a **non-inferiority test** (explained below).


## Null Hypothesis Significance Testing (NHST)

At a high level, the **NHST** workflow is:

1. **Assume what you *don’t* want to see** — this is the **null hypothesis**.  
   - Example in medicine: *“the drug has no effect.”*  
   - Example here: *“the new experience increases abandonment.”*
2. **Run the experiment** and compute a test statistic.
3. **Ask:** *If the null were true, how likely is it that we would observe a result at least this extreme?*  
   - If that probability (the **p-value**) is very low — e.g., below a conventional threshold such as 5% — we **reject the null**.

Two immediate caveats:
- Rejecting the null does **not** prove the opposite is true; it only says the data would be unlikely *if* the null were correct.
- “Unlikely enough” is arbitrary — thresholds like 5% are conventions, not laws of nature.

A key point: NHST computes **$P(\text{data} \mid \text{hypothesis})$**.  
Later we’ll see that the Bayesian approach instead computes **$P(\text{hypothesis} \mid \text{data})$** — a fundamentally different quantity.

---

### Modeling Conversion as Random Variables

The abandonment or conversion of a UX flow can be modeled with **Bernoulli random variables**:

- $X_C$ for the control experience
- $X_A$ for a new variant $A$

A Bernoulli variable takes only two values: success/failure, convert/abandon, etc.  
Each user who sees a page gives one draw from one of these variables.

We assume both have the same codomain:

$$
\mathcal{X}_C = \mathcal{X}_A = \{0,1\}
$$

where **1 = convert** (user finishes the intended action, e.g., creating a passkey) and **0 = abandon**.  
Technical failures are treated as *success* here because the user attempted the action.

---

### Sample Proportions

NHST usually works with **sample proportions**, the average of $n$ Bernoulli draws:

$$
\hat{p}_C = \frac{1}{n}\sum_{i=1}^n X_{C_i},
\quad
\hat{p}_A = \frac{1}{n}\sum_{i=1}^n X_{A_i}.
$$

Each $\hat{p}$:

- Is a random variable taking values $\{0,\tfrac1n,\tfrac2n,\ldots,1\}$.
- Is also an **estimator** of the true expected value $p = E[X]$.  
  By the law of large numbers, $\hat{p} \to p$ as $n$ grows.

(Statisticians use a “hat” to denote an estimator.)

Formally, an estimator maps $n$ realizations of $X$ ($\mathcal{X}^n$) to a real number:

$$
\hat{p}: \mathcal{X}^n \to [0,1].
$$

Because it is the mean of $n$ Bernoulli variables, $\hat{p}$ follows a **binomial** distribution that becomes approximately **Gaussian** when $n$ is large.

---

### Variance and Standard Deviation of a Sample Proportion

For a single Bernoulli $X$:  
$$
\mathrm{Var}(X) = p(1-p).
$$

For the sample proportion:
$$
\mathrm{Var}\!\left(\tfrac1n \sum_{i=1}^n X_i\right)
= \tfrac1{n^2} n p(1-p)
= \tfrac{p(1-p)}{n}.
$$

$$
\boxed{\mathrm{Var}(\hat{p}) = \frac{p(1-p)}{n}}
$$

The square root of this variance is the **standard error** — a quantity we’ll use later.

$$
\boxed{SE = SD(\hat{p}) = \sqrt{\frac{p(1-p)}{n}}}
$$

---

### Difference in Proportions

We’ll often look at the **difference** between variant and control:

$$
\hat{\Delta} = \hat{p}_A - \hat{p}_C
$$

This estimates the true difference

$$
\Delta = p_A - p_C.
$$

---

### Hypotheses

- **Null Hypothesis $H_0$** — the “bad” scenario we want to reject:  
  the new UX **degrades** conversion by at least some small amount $\epsilon$ we consider unacceptable (e.g., $3\%$):

  $$
  H_0: E[\Delta] \le -\epsilon
  $$

- **Alternative Hypothesis $H_1$** — the new UX is **not worse** than control (possibly better):

  $$
  H_1: E[\Delta] > -\epsilon
  $$

- **Boundary Hypothesis** — used in test construction:  
  assume the difference is exactly at the acceptable degradation limit:

  $$
  E[\Delta] = -\epsilon
  $$


#### Numerical Example

For a concrete example, we define the following counts and quantities for each experience:

- $n_C$ — number of visitors in the **control** group  
- $x_C$ — number of **conversions** observed in the control group

- $n_A$ — number of visitors in the **variant** group  
- $x_A$ — number of **conversions** observed in the variant group

- $\hat{\Delta}_{\mathrm{obs}}$ — the **observed difference** in conversion proportions between variant and control

- $\epsilon$ — the **acceptable degradation margin**, i.e., the smallest decrease in conversion we are willing to tolerate for the new variant




In [ ]:
control_group_conversion_rate = 0.2 # based on historical data
nC = 7000
xC_observed = nC * control_group_conversion_rate
nA = 150
xA_observed = 33
epsilon = 0.03
alpha = 0.05
hatpC_observed = xC_observed / nC
hatpA_observed = xA_observed  / nA
hatDelta_observed = hatpA_observed - hatpC_observed

print(f"Realization of difference in conversion rate estimator: {hatDelta_observed:.4f}")
print(f"Control group realization of conversion rate estimator: {hatpC_observed:.4f}")
print(f"Treatment group realization of conversion rate estimator: {hatpA_observed:.4f}")

### Standard Deviation of the Estimator $\hat{\Delta}$ (a.k.a. Standard Error in Frequentist Statistics)

In NHST, the first step is to **estimate the standard deviation** of the estimator $\hat{\Delta}$ (often called the **standard error**, SE).  
We then compare the observed difference in proportions from the experiment to this estimated variability to decide whether the observed effect is “far enough” from what we would expect under the null hypothesis $H_0$.

This is a key pain point for NHST:

- We **do not know** the true standard deviation — it depends on the unknown underlying conversion probabilities.
- Frequentist methods therefore use the **plug-in principle**: estimate the unknown variance by “plugging in” the sample estimates (the data you just observed).

But note the circularity:

1. We want to know if the data are unusual under $H_0$.
2. To measure “unusual,” we need the standard error assuming $H_0$.
3. SE depends on the unknown true rates, so we **plug in** $\hat{p}$ (from the data!).
4. We then use this data-derived SE to judge whether the data are unusual.

It’s like saying: *“Use my one measurement to tell me how variable my measurements are, then use that to decide if my measurement is surprising.”*

Frequentists accept this because:

- **Long-run frequency view:** if we repeated the procedure many times, it would have correct average properties.
- **Pragmatism:** it’s computable from one experiment.
- **Simulation evidence:** works “reasonably well” for moderate $n$, though “reasonably well” is debatable (see the *replication crisis*).
- **Framework limitation:** in classical stats, parameters are fixed unknowns, so no natural way to treat them as random.

---

### Common Plug-In Approaches (a.k.a. “Standard Error Hacks”)

#### 1. Wald **Pooled** Standard Error (for a “No Effect” Hypothesis)

If an hypothesis is **no effect** ($p_A = p_C$), we can pool data from control and variant, because under the hypothesis they are assumed to come from the same distribution.

Realizations of sample proportions are:

$$
\hat{p}_A = \frac{x_A}{n_A}, \qquad
\hat{p}_C = \frac{x_C}{n_C}.
$$

If "no effect" as true (no difference), a pooled estimator is:

$$
\hat{p}_{\text{pool}} = \frac{x_A + x_C}{n_A + n_C}.
$$

Variance of the difference between two independent proportions is the sum of their variances:

$$
\mathrm{Var}(\hat{p}_A - \hat{p}_C)
= \mathrm{Var}(\hat{p}_A) + \mathrm{Var}(\hat{p}_C).
$$

Plugging in the pooled estimate:

$$
\mathrm{Var}(\hat{p}_A - \hat{p}_C)
= \hat{p}_{\text{pool}}(1-\hat{p}_{\text{pool}})
\left(\frac{1}{n_A}+\frac{1}{n_C}\right).
$$

So the **pooled standard error** is:

$$
\boxed{\text{WaldPooled SE} =
\sqrt{\hat{p}_{\text{pool}}(1-\hat{p}_{\text{pool}})
\left(\frac{1}{n_A}+\frac{1}{n_C}\right)} }
$$

This is the classic **standard error of the difference between two independent proportions**.

Once SE is computed, we calculate a **z-score** (number of SEs the observed difference is away from the null value 0):

$$
z = \frac{\hat{p}_A - \hat{p}_C}{\text{SE}}.
$$

---

#### 2. Wald **Unpooled** Standard Error (for Non-Inferiority)

If the null is **non-inferiority** (allowing a margin $\epsilon$), we **cannot** assume $p_A = p_C$, so we don’t pool.

Instead we sum the individual variances (using the plug-in trick separately for each group):

$$
\widehat{\text{WaldUnpooled SE}} =
\sqrt{\frac{\hat{p}_A(1-\hat{p}_A)}{n_A} +
      \frac{\hat{p}_C(1-\hat{p}_C)}{n_C}}.
$$

Ideally, the true $p_A$ and $p_C$ should be used here, but we don’t know them — so we substitute $\hat{p}_A$ and $\hat{p}_C$.  
This works but can be inaccurate if sample sizes are small or rates are extreme.

---

#### 3. Newcombe / Score-Based (Wilson)

- **Better coverage** than Wald, especially with imbalanced sample sizes or very high/low $p$.
- Still closed-form and relatively easy to compute.
- Still uses plug-in estimates and suffers from the same “single-sample” limitation.

*(Not detailed here — but recommended over Wald when possible.)*

---

#### 4. Miettinen–Nurminen

- Widely used in **clinical trials** and regulated industries (e.g., FDA guidance).
- Provides improved accuracy for non-inferiority tests.
- However: mathematically complex, still plug-in based, and still fundamentally relies on one sample.

---

### Summary

Frequentist methods **must estimate variance from the data itself** — leading to circularity and potential miscalibration, especially for small samples or edge cases.  
Later we’ll see how Bayesian methods avoid this by modeling uncertainty about the true conversion rates directly.


Numerical examples

In [ ]:
pooled_proportion = (xC_observed + xA_observed) / (nC + nA)
wald_pooled_SE = (pooled_proportion * (1 - pooled_proportion) * (1/nC + 1/nA))**0.5
print(f"Wald Pooled Standard Error: {wald_pooled_SE:.4f}")
wald_unpooled_SE = ((hatpC_observed * (1 - hatpC_observed) / nC) + (hatpA_observed * (1 - hatpA_observed) / nA))**0.5
print(f"Wald Unpooled Standard Error: {wald_unpooled_SE:.4f}")

#### Probability of False Positive, *p*-value, Significance Level $\alpha$ (sometimes called “confidence”), and Critical Value

Once we have an estimate of the standard error $SE$ (the standard deviation of $\hat{\Delta}$), NHST assumes a **sampling distribution** for the estimator under the null hypothesis $H_0$.

The idea is:

- If we know (or assume) the **mean** and **standard deviation** of $\hat{\Delta}$ under $H_0$,  
- we can model it with a known probability distribution and calculate how likely any observed result is.

Although the true process is discrete (binomial), in practice we often approximate it by a **normal (Gaussian)** distribution. This is mathematically simpler and is a good approximation for moderate or large sample sizes.

---

##### Using the “Boundary” Case

The null hypothesis  for "non inferiority" is technically an inequality:

$$
H_0: E[\Delta] \le -\epsilon.
$$

But to get a single distribution to work with, we typically use the **boundary value**:

$$
E[\Delta] = -\epsilon.
$$

Why?  
- The boundary is the **least favorable** case for rejecting $H_0$.  
- If we can reject $H_0$ at $E[\Delta] = -\epsilon$, then we would also reject any stronger null (mean further left).  
- This makes the test conservative.

So under $H_0$, we model $\hat{\Delta}$ as:

$$
\hat{\Delta} \sim N(\mu, \sigma)
$$

with

$$
\mu = -\epsilon, \qquad \sigma = SE.
$$

---

##### Computing the *p*-Value

The **p-value** is the probability (under $H_0$) of observing a result **as extreme or more extreme** than what we got, in the direction of the alternative $H_1$.

In this one-sided non-inferiority test, that means the right tail probability:

$$
p\text{-value} = P_{H_0}\big[\hat{\Delta} \ge \hat{\Delta}_{\text{obs}}\big]
= \int_{\hat{\Delta}_{\text{obs}}}^{+\infty} 
\frac{1}{\sqrt{2\pi}\,\sigma}
\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)\,dx.
$$

This tail integral is the **survival function** of the normal distribution.

Using the standard normal CDF $\Phi$:

$$
p\text{-value} = 1 - \Phi\!\left(\frac{\hat{\Delta}_{\text{obs}}-\mu}{\sigma}\right).
$$

---

##### Significance Level $\alpha$ and Critical Value

- We choose a **significance level** $\alpha$ (often $0.05$).  
- If $p\text{-value} \le \alpha$, we reject $H_0$ — our result is unlikely under the null.

The **critical value** $c$ is the smallest observed difference that would lead to rejection at level $\alpha$. It is obtained by inverting the right-tail probability:

$$
c = \mu + \sigma \,\Phi^{-1}(1 - \alpha).
$$

Any observed $\hat{\Delta}_{\text{obs}} \ge c$ yields $p\text{-value} \le \alpha$ and thus rejects $H_0$.

---

Below we’ll compute the p-value and critical value explicitly and visualize how the right tail behaves.


In [ ]:
SE_H0 = wald_unpooled_SE
from scipy.stats import norm

mu_H0 = -epsilon    # mean
sigma_HO = SE_H0  # standard deviation
x = hatDelta_observed  # value to evaluate

# Survival function P(X > x)
p_value = norm.sf(x, loc=mu_H0, scale=sigma_HO)
print(f'p-value (one-sided) for an observed proportion at {hatDelta_observed:.4f}: {p_value:.4f}')

# inverse survival function to find critical value for given p-value
critical_value = norm.isf(alpha, loc=mu_H0, scale=sigma_HO)
print(f"Critical value for our alpha cutoff value at {alpha:.4f}: {critical_value:.4f}")



So with the value we chose as an example for the sample, the p-value is a bit more than 7%, so that's not unlikely enough if we picked 5% as the alpha cutoff point. So we would "fail to reject", meaning we can't say much, we don't know whether we can claim that the new CX does not cause some unwanted degradation.

Note that this is because the sample is tiny. We picked it small on purpose as this is a realistic situation when launching a new feature; for various reasons including unresolved bugs you usually just put a tiny bit of traffic on the new feature. But those small samples are often not sufficient to make a decision based on NHST. This is due to the way the standard error is computed; it needs more data to deliver some insights. In this example we would need our observed proportion to get to 0.026 or better to reject the null.

In [ ]:
# Plot the Gaussian N(mu, sigma) and shade the right-tail area beyond x

# Use previously defined values; recompute to be robust
SE_H0 = wald_unpooled_SE
mu_H0 = -epsilon
sigma_HO = SE_H0
x0 = hatDelta_observed

# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H0 - 6 * sigma_HO
right = mu_H0 + 6 * sigma_HO
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H0, scale=sigma_HO)

# Right-tail probability
p = norm.sf(x0, loc=mu_H0, scale=sigma_HO)

# Critical value at significance alpha (one-sided)
crit_x = norm.ppf(1 - alpha, loc=mu_H0, scale=sigma_HO)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density")

# Shade right tail
mask = xs >= x0
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, label=f"Right tail p = {p:.4g}")

# Vertical line at observed x
ax.axvline(x0, color="C1", ls="--", lw=1.5, label=f"observed delta = {x0:.4f}")

# Vertical line at critical value
ax.axvline(crit_x, color="C2", ls="-.", lw=1.5, label=f"critical value c = {crit_x:.4f}")

# Vertical line at the mean for the null hypothesis H0
ax.axvline(-epsilon, color="k", ls=":", lw=1.5, label=f"mean under H0 = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H0:.4f}, σ={sigma_HO:.4f}) — Right-tail beyond x")
ax.set_xlabel("Δ (difference in proportions) under the null H0")
ax.set_ylabel("Probability density under H0")
ax.legend(loc="best")
ax.grid(True, ls=":", alpha=0.5)
plt.tight_layout()
# plt.show(), display handled by notebook



#### Traditional Presentation Using *z*-Scores

Another common way to compute the p-value in NHST is to **standardize** the observed statistic rather than work directly with $\hat{\Delta}_{\mathrm{obs}}$.

The idea:

- Convert our Gaussian with mean $\mu$ and standard deviation $\sigma$ into a **standard normal** $N(0,1)$.
- This is done by the familiar **$z$-score transformation**:

$$
Z = \frac{X - \mu}{\sigma}.
$$

For our non-inferiority test:

$$
Z_{\mathrm{NI}}
= \frac{\hat{\Delta} - E[\Delta]_{H_{\text{boundary}}}}{SE}
= \frac{\hat{\Delta} - (-\epsilon)}{SE}
= \frac{\hat{\Delta} + \epsilon}{SE}.
$$

Thus $Z_{\mathrm{NI}}$ follows approximately a standard normal $N(0,1)$ under $H_0$.

---

##### Tail Probability in Standard Normal Form

We want the probability (under $H_0$) of observing something at least as extreme as our sample:

$$
P\big[\hat{\Delta} \ge \hat{\Delta}_{\mathrm{obs}}\big]
= P\!\left[\frac{\hat{\Delta} + \epsilon}{SE}
       \ge \frac{\hat{\Delta}_{\mathrm{obs}}+\epsilon}{SE}\right].
$$

This is simply the **right-tail** probability of a standard normal:

$$
\int_{Z_{\mathrm{NI}}}^{+\infty} 
\frac{1}{\sqrt{2\pi}}\,e^{-z^2/2}\,dz.
$$

Using the standard normal survival function gives the same p-value as before — this is just the “classical” z-score framing that many NHST tutorials use.


In [ ]:
zni = (hatDelta_observed + epsilon) / SE_H0
p_zni = norm.sf(zni)
print(f"z_NI: {zni:.4f}, p-value: {p_zni:.4f}")

This is hte same pvalue, just a different way to compute the integral

#### False Positive (a.k.a. Type I Error)

In this NHST setup, the **alpha**  represents the cutoff for the decision rule (lower means we reject H0, it is the condtional probability of rejecting H0 while it is actually true. In this setup the rejection is considered a "postive" as H0 is "what we don't want to see", so that is concluding “no unacceptable degradation” while there is actually a nasty one.

a more formal way of formualting the pvalue is p(Reject H0 | H0 is true)


By setting the significance level $\alpha = 0.05$, we accept a **5% risk** of making this wrong decision when there is a degration. But note that this is a frequentist definition: if we ran the experiment many times,  on average we would incorreclty reject 5% of the time in this case. However tt does assign any actual probablity to our current decision/experiment (Bayesian method do, see below).

It also says nothing about the "effect size", that is about how much "non-degradation/improvement" we may have. For non-inferiority it does not matter too much but for superiority we would want to know, also something Bayesian approach can do more directly.


#### False Negative (Type II Error), Power, and Sample Size

The **false negative** — is failing to reject $H_0$ when the alternative $H_1$ is actually true. 

In the context of a **non-inferiority test**, a false negative means:

> We fail the test (do **not** reject $H_0$) even though the new UX is truly **non-inferior** (as good or better than the old one). Usually it means we woud need to keep on gathering data until the test has more power to detect something (see below)

---

##### Choosing an Effect Size Under $H_1$

Just as with the Type I error calculation, we need to pick an expected value for the difference $\Delta$ — but this time **under $H_1$**.

- In practice, we must choose a **single reference value** to center the alternative distribution.  
- A common (and pragmatic) choice is the **minimum effect size we care to detect** — often set to $E[\Delta] = 0$ (meaning *no difference* between variant and control).  
  - If the variant is truly “no worse” (Δ = 0), the test should reject $H_0$ most of the time.

This choice is somewhat **arbitrary** and reflects a **business decision**: “How small of a difference do we consider acceptable to detect?”

---

##### Modeling Under $H_1$

If we assume the variant is truly **no worse** (Δ = 0), we can pool samples to estimate the standard error (since under $H_1$ we’re treating them as coming from the same distribution):

$$
SE_{H_1}
= \text{WaldPooled SE}
= \sqrt{\hat{p}_{\mathrm{pool}}
(1-\hat{p}_{\mathrm{pool}})
\left(\tfrac{1}{n_C}+\tfrac{1}{n_A}\right)}.
$$

We then compare this **alternative distribution** (mean = 0, std = $SE_{H_1}$) to the **critical value** $c$ that was already set by the significance level $\alpha$.

---

##### Beta and Power

- **$\beta$ (Type II error)** = probability that the observed statistic falls **below the critical value** when the true mean is the one we chose under $H_1$ (e.g., Δ = 0).  
- **Power** = $1-\beta$ = probability of **correctly rejecting** $H_0$ when the variant is truly non-inferior. Another way to say this is: knowing the property we care about is really there (non-inferiority), what is our probability of detecting it. In machine learning and search queries analysis, Power is also called "recall", that is if it is really there can we predict or find it.

Graphically:  
- The null distribution is centered at $-\epsilon$ (our boundary).  
- The alternative distribution is centered at $0$ (no degradation).  
- $\beta$ is the area of the alternative distribution **to the left of the critical value**.

With the numerical example we picked (on purpose, small sample and very unbalanced between control and variant) the power is not good at all (see below). The conventional target most tests try to achieve is 80% power; we are only at about 21%.


In [ ]:
SE_H1 = wald_pooled_SE
mu_H1 = 0
sigma_H1 = SE_H1
x = critical_value
beta = norm.cdf(x, loc=mu_H1, scale=sigma_H1)
print(f"Observed probability of false negative a.k.a β a.k.a type 2 errors,  at critical value : {beta:.4f}")
power = 1 - beta
print(f"Observed Power (1 - β): {power:.4f}")


In [ ]:
# Domain for plotting (±6σ around the mean, clipped to reasonable bounds)
left = mu_H1 - 6 * sigma_H1
right = mu_H1 + 6 * sigma_H1
xs = np.linspace(left, right, 1000)
pdf = norm.pdf(xs, loc=mu_H1, scale=sigma_H1)

# Plot
fig, ax = plt.subplots(figsize=(7.5, 4.5))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.plot(xs, pdf, color="C0", lw=2, label="Density under H₁")

# Shade left tail (Type II error region)
mask = xs <= critical_value
ax.fill_between(xs[mask], pdf[mask], color="C1", alpha=0.35, 
                label=f"Type II error β = {beta:.4g}")

# Vertical line at critical value
ax.axvline(critical_value, color="C2", ls="-.", lw=1.5, 
           label=f"critical value c = {critical_value:.4f}")

# Vertical line at observed delta
ax.axvline(hatDelta_observed, color="C1", ls="--", lw=1.5, 
           label=f"observed delta = {hatDelta_observed:.4f}")

# Vertical line at the mean under H1
ax.axvline(mu_H1, color="k", ls=":", lw=1.5, 
           label=f"mean under H₁ = {mu_H1:.4f}")

# Vertical line at the H0 boundary (for reference)
ax.axvline(-epsilon, color="gray", ls=":", lw=1.5, alpha=0.7,
           label=f"H₀ boundary = {-epsilon:.4f}")

# Decorations
ax.set_title(f"Normal(μ={mu_H1:.4f}, σ={sigma_H1:.4f}) — Type II Error (β) and Power")
ax.set_xlabel("Δ (difference in proportions) under H₁")
ax.set_ylabel("Probability density under H₁")
ax.legend(loc="best", fontsize=9)
ax.grid(True, ls=":", alpha=0.5)

# Add text annotation for power
power_text = f"Power = 1 - β = {power:.4f}"
ax.text(0.98, 0.95, power_text, transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()


---

##### Designing for Target Power

If we want to achieve a **target power** — commonly 80% (so $\beta = 0.2$) —  
we can **solve for the required sample size** (embedded in $SE$).

- Larger $n$ → smaller $SE$ → distributions separate more clearly → higher power.
- This is the usual **sample size calculation** step when planning an A/B test.

In practice, one:
1. Fixes $\alpha$ (e.g., 0.05).
2. Chooses the minimum effect size of interest (e.g., $\Delta=0$ for non-inferiority).
3. Sets desired power (e.g., 80%).
4. Solves for $n_C$ and $n_A$ to achieve that power given the pooled variance.

The code to solve to get a target beta can be developed easily, but since we will favor Bayesian approaches which can work with small samples, we will leave that TBD.



---

##### NHST Confidence Interval (CI)

NHST has also a notion of Confidence Interval but it is not what most people think it means. It does not say anything about the hypothesis or whether the observed value is close to the truth, or whether rejecting H0 is true or not. It is computed without using any of the hypotheses; it is just using the "plugged in" standard deviation SE (derived from the observation, with all the caveats we discussed) and says: if the standard deviation (standard error) was really SE, our sample would end up in the interval [a,b] 95% of the time. Not useful to make any decisions directly. Some people use it indirectly by looking for overlaps between the interval and some key values, but that's just another way to do the p-value analysis as above, or "picking the best variant" below. The p-value computation is the same computation really and the mainstream way of doing it for NHST practitioners.


### Bayesian Approach

In contrast to NHST, the **Bayesian approach** is conceptually simpler:

- Instead of fixing two specific expected values for $\Delta$ (one under $H_0$ and one under $H_1$),
- We treat the **true conversion difference** $E[\Delta]$ itself as an **unknown random variable** and reason about its entire probability distribution.

This lets us quantify directly how likely any value of $\Delta$ is, given both prior knowledge and the data we observe.

The typical workflow is

1. Pick a prior belief, express it as a Beta distribution
2. Run the experiment
3. Use Bayes theorem to update the prior belief into a posterior belief
4. Rinse and repeat as this is one of the strengths of the method; the posterior belief can be used as a new prior before running more experiments that will firm it up, if needed and with perfect mathematical rigor unlike NHST which does not allow peeking or stopping early because of the way sampling works.

---

#### Using the Beta Distribution for Our Prior Belief

For experiments based on **Bernoulli trials** (success/failure, convert/abandon, etc.), the most convenient way to model our **prior belief** about a conversion rate is the **Beta distribution**.

> ⚠️ Don’t confuse this **Beta** with the “$\beta$” from NHST (Type II error).  
> Here, *Beta* is the name of a probability distribution.

The Beta distribution:

- Is defined on the interval $[0,1]$, making it perfect for modeling a **probability**.
- Has two shape parameters, $\alpha$ and $\beta$, which control how strongly it reflects our prior knowledge.

Some examples of possible priors:

- **Uninformative prior:** $ \mathrm{Beta}(1,1) $ — essentially a uniform distribution, expressing “we know nothing.”
- **Weakly informative prior:** centered roughly around 17–20% but without being very sure

Here are a few graphical example of the Beta distribution for various values of $\alpha$ and $\beta$:


In [ ]:
x = np.linspace(0, 1, 1000)

# Four cases:
cases = [
    ("Uninformative (flat)", ("single", (1, 1))),
    ("Weakly informative (centered, high entropy)", ("single", (3, 12))),   # mean=0.2
    ("Strong conviction (centered, low entropy)", ("single", (200, 800))),  # mean=0.2
    ("Bi-modal (mixture of Betas)", ("mixture", ((5, 20, 0.5), (20, 5, 0.5))))
]

fig, axes = plt.subplots(2, 2, figsize=(10, 7), sharex=True, sharey=True)

for ax, (title, (kind, params)) in zip(axes.ravel(), cases):
    if kind == "single":
        a, b = params
        y = beta_dist.pdf(x, a, b)
        mean = a / (a + b)
        ci_low, ci_high = beta_dist.ppf([0.025, 0.975], a, b)

        ax.plot(x, y, color="C0", lw=2)
        ax.fill_between(x, y, color="C0", alpha=0.12)
        ax.axvline(mean, color="k", ls=":", lw=1.2, label=f"mean={mean:.3f}")
        ax.axvline(ci_low, color="C1", ls="--", lw=1, label="95% CI")
        ax.axvline(ci_high, color="C1", ls="--", lw=1)
        ax.set_title(f"{title}\nBeta(α={a}, β={b})", fontsize=10)
        ax.legend(fontsize=8, loc="best")
    else:
        # Mixture of two Betas: (a1, b1, w1), (a2, b2, w2)
        (a1, b1, w1), (a2, b2, w2) = params
        y = w1 * beta_dist.pdf(x, a1, b1) + w2 * beta_dist.pdf(x, a2, b2)
        m1, m2 = a1 / (a1 + b1), a2 / (a2 + b2)
        mix_mean = w1 * m1 + w2 * m2

        ax.plot(x, y, color="C0", lw=2, label="mixture pdf")
        ax.fill_between(x, y, color="C0", alpha=0.12)
        # Component means
        ax.axvline(m1, color="C2", ls="--", lw=1, label=f"mean₁={m1:.2f}")
        ax.axvline(m2, color="C3", ls="--", lw=1, label=f"mean₂={m2:.2f}")
        # Mixture mean
        ax.axvline(mix_mean, color="k", ls=":", lw=1.2, label=f"mixture mean={mix_mean:.2f}")
        ax.set_title(f"{title}\n{w1:.1f}·Beta({a1},{b1}) + {w2:.1f}·Beta({a2},{b2})", fontsize=10)
        ax.legend(fontsize=8, loc="best")

for ax in axes[-1]:
    ax.set_xlabel("p")
for ax in axes[:, 0]:
    ax.set_ylabel("density")

fig.suptitle("Four Beta Priors: flat, weakly centered, strongly centered, bi-modal", fontsize=12)
plt.tight_layout(rect=[0, 0, 1, 0.95])

The formal definition of the **Beta distribution** looks a bit intimidating; both the numerator and denominator are related to the probability (binomial) when the probability of having m successes out of n trials (binomial) where the basic event probability is set to x.
Intuition: $\alpha - 1$ and $\beta - 1$ act like prior pseudo-counts of successes m and failures n-m. After observing data, you add the real counts.
Special case (uniform prior): Beta(1,1) ⇒ posterior Beta(m + 1, n - m + 1).

$$ 

f(x, \alpha, \beta) = \frac{x^{\alpha-1}(1-x)^{\beta-1}}{B(\alpha, \beta)}

$$

with B being the **Beta function B** defined as normalizing constant:

$$

B(\alpha, \beta) = \int_0^1 t^{\alpha-1} (1-t)^{\beta-1} dt

$$

There is no easy way to work with this formula "by hand" and that is one of the reasons Bayesian approaches were historically impractical as you cannot work with those "by hand" like with a Gaussian function. But now that we have stats packages in Python it is trivial to use. Here are some examples of the shapes it can take by picking different values of $\alpha$ and $\beta$.




#### Conservative Approach: Assuming We Know Nothing (Non-Informative Prior)

For the **variant A**, suppose we start with a **non-informative prior** — meaning we have no knowledge about the conversion rate $p_A$.  
We assume $p_A$ could be anywhere between $0$ and $1$ with equal probability.  
This is modeled by the Beta distribution:

$$
\mathrm{Beta}(1,1)
$$

— which is just a **uniform prior** (flat line) on $[0,1]$.

---

##### Posterior After Observing Data

After running the experiment with:

- $n_A$ = number of trials (users shown variant A),
- $x_A$ = number of successes (conversions),

the **posterior** distribution for $p_A$ — thanks to the conjugacy of the Beta with the Bernoulli likelihood  is:

$$
\mathrm{Beta}(x_A+1,\; n_A - x_A + 1).
$$

This follows directly from **Bayes’ theorem** and the properties of the Beta distribution.

---

##### Expected Value of the Posterior

For any $\mathrm{Beta}(\alpha,\beta)$ distrib


In [ ]:
expected_value_posterior = (xA_observed + 1) / (nA + 2)
print(f"Expected value of posterior distribution for p_A: {expected_value_posterior:.4f}")

#### Credible Intervals and Visualizing Prior vs. Posterior

A **credible interval** answers: *“Given the data and our prior, what range of parameter values has (say) 95% posterior probability?”*   


Note that this is completely different than the confidence interval of NHST althoug it sounds kind of the same, it is not, this give us a direct probability of the truth of our hypothesis we can say "we know that the true conversion rate is between a and b with x% probability. We can pick whatever x we want and we will get an interval

So let's say we use **equal-tailed** credible intervals (quantiles at 2.5% and 97.5%).


- **Prior** (uninformative): $\mathrm{Beta}(1,1)$  
- **Posterior** after observing $x_A$ conversions out of $n_A$:  
  $\alpha_A = x_A+1,\; \beta_A = n_A - x_A + 1$

We can compute these intervals and plot how the **posterior** updates our belief compared with the **prior**.


In [ ]:
# Posterior parameters
alpha = xA_observed + 1
beta_param = nA - xA_observed + 1

# Ensure we use the beta distribution from scipy.stats
from scipy.stats import beta as beta_dist

# Compute 95% credible interval (2.5th and 97.5th percentiles)
p_L = beta_dist.ppf(0.025, alpha, beta_param)
p_U = beta_dist.ppf(0.975, alpha, beta_param)

# Output the result
print(f"95% Credible Interval for p: [{p_L:.4f}, {p_U:.4f}]")

So we know with 95% chance of being true that the true converstion rate is between 16.13% and 29.30%. Here is a visualization

In [ ]:
# Visualize non-informative prior vs posterior
x_range = np.linspace(0, 0.5, 1000)
prior_noninformative_pdf = beta_dist.pdf(x_range, 1, 1)  # Beta(1,1) - uniform
posterior_noninformative_pdf = beta_dist.pdf(x_range, alpha, beta_param)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_range, prior_noninformative_pdf, 'b--', lw=2, label='Prior: Beta(1, 1) - Non-informative')
ax.plot(x_range, posterior_noninformative_pdf, 'r-', lw=2, label=f'Posterior: Beta({alpha:.1f}, {beta_param:.1f})')

# Mark the non-inferiority boundary
ax.axvline(control_group_conversion_rate - epsilon, color='k', ls=':', lw=1.5, 
           label=f'Non-inferiority boundary = {control_group_conversion_rate - epsilon:.2f}')

# Mark the control conversion rate
ax.axvline(control_group_conversion_rate, color='g', ls=':', lw=1.5, 
           label=f'Control conversion rate = {control_group_conversion_rate:.2f}')

# Shade the 95% credible interval
mask = (x_range >= p_L) & (x_range <= p_U)
ax.fill_between(x_range[mask], posterior_noninformative_pdf[mask], alpha=0.3, color='red', 
                label='95% Credible Interval')

ax.set_xlabel('Conversion rate p_A', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Non-informative Prior vs Posterior Distribution', fontsize=14)
ax.legend(loc='best')
ax.grid(True, ls=':', alpha=0.5)
plt.tight_layout()
plt.show()

In this first version we assumed that "we know nothing" prior to the experiment, which is not realistic. We know we added a page with an extra click, so unless there is a serious bug the conversion should be "around" the control which has a historical mean of 20%, but we want to be open to the possibility of small deviation around this historical value. We can do this with what is known as a weakly informative prior.

#### Weakly Informative Prior Using Historical Data

Instead of using a completely non-informative prior $\mathrm{Beta}(1,1)$, we can incorporate **historical knowledge**.

Suppose we know the **control group** conversion rate is about **0.20**, and we want to test for **non-inferiority** with a margin $\epsilon = 0.03$.

For the variant, we choose a prior **centered** at the historical mean 0.20 but we want this prior to have **high entropy** (wide uncertainty) so it does not dominate the data.  

A **Beta** distribution with a modest $\alpha$ and $\beta$ can be informative about the center while still uncertain.

For a $\mathrm{Beta}(\alpha,\beta)$ distribution:

$$
\mu = \frac{\alpha}{\alpha+\beta}
$$

- Smaller values of $\alpha$ and $\beta$ give a **wider** (more uncertain) prior.

Let’s pick $\alpha = 20$ and solve for $\beta$ so the mean is $0.2$, Then recompute the posterior distribution but with this prior


In [ ]:
# Informative prior parameters
target_prior_mean = control_group_conversion_rate  # 20%
alpha_prior = 20 # Small value for high entropy
beta_prior = (alpha_prior / target_prior_mean) - alpha_prior  # Solve for beta given mean

print(f"Prior: Beta({alpha_prior:.2f}, {beta_prior:.2f})")
print(f"Prior mean: {alpha_prior / (alpha_prior + beta_prior):.4f}")
print(f"Prior variance: {(alpha_prior * beta_prior) / ((alpha_prior + beta_prior)**2 * (alpha_prior + beta_prior + 1)):.6f}")

# Posterior parameters after observing data
alpha_posterior = xA_observed + alpha_prior
beta_posterior = (nA - xA_observed) + beta_prior

print(f"\nPosterior: Beta({alpha_posterior:.2f}, {beta_posterior:.2f})")
posterior_mean = alpha_posterior / (alpha_posterior + beta_posterior)
print(f"Posterior mean: {posterior_mean:.4f}")

# Compute probability that variant is non-inferior (p_A > p_C - epsilon)
# This is P(p_A > 0.17) under the posterior
non_inferiority_threshold = control_group_conversion_rate - epsilon
prob_non_inferior = 1 - beta_dist.cdf(non_inferiority_threshold, alpha_posterior, beta_posterior)

print(f"\nProbability that variant is non-inferior: {prob_non_inferior:.4f}")
print(f"This means there's a {prob_non_inferior*100:.2f}% probability that the variant conversion rate is above {non_inferiority_threshold:.2f}")

So here unlike the NHST, thanks to adding a reasonable prior that uses our understanding of how the CX for passkey is built, we end up having something actionable. We just have an above 95% probability of being over the cutoff so we are non-inferior, good to go.

Here is a diagram of this prior and the posterior after observing data $(x_A, n_A)$ for the variant, and various credible intervals. For non-inferiority we don't even need the credible interval (see next).


In [ ]:
# Compute probability that variant is non-inferior (p_A > p_C - epsilon)
# This is P(p_A > 0.17) under the posterior
prob_non_inferior = 1 - beta_dist.cdf(control_group_conversion_rate - epsilon, 
                                       alpha_posterior, beta_posterior)
print(f"Probability that variant is non-inferior: {prob_non_inferior:.4f}")
print(f"This means there's a {prob_non_inferior*100:.2f}% probability that the variant conversion rate is above {control_group_conversion_rate - epsilon:.2f}")

In [ ]:
# Prior vs Posterior with non-inferiority tail area (P(p_A > p_C - ε))

# Prereqs: control_group_conversion_rate, epsilon, alpha_prior, beta_prior,
#          alpha_posterior, beta_posterior, beta_dist, np, plt

threshold = control_group_conversion_rate - epsilon

# Compute probabilities (posterior and prior) for reference
prob_non_inferior_post = beta_dist.sf(threshold, alpha_posterior, beta_posterior)
prob_non_inferior_prior = beta_dist.sf(threshold, alpha_prior, beta_prior)

# Plot range focused around the relevant region
x_min = max(0.0, threshold - 0.12)
x_max = min(1.0, threshold + 0.28)
x_range = np.linspace(x_min, x_max, 1200)

prior_pdf = beta_dist.pdf(x_range, alpha_prior, beta_prior)
post_pdf  = beta_dist.pdf(x_range, alpha_posterior, beta_posterior)

fig, ax = plt.subplots(figsize=(10, 6))

# Curves
ax.plot(x_range, prior_pdf, color="#1f77b4", lw=2, ls="--",
        label=f"Prior Beta({alpha_prior:.2f}, {beta_prior:.2f})")
ax.plot(x_range, post_pdf, color="#d62728", lw=2.5,
        label=f"Posterior Beta({alpha_posterior:.2f}, {beta_posterior:.2f})")

# Non-inferiority boundary
ax.axvline(threshold, color="k", ls=":", lw=1.8,
           label=f"Non-inferiority boundary = {threshold:.2f}")

# Shade posterior tail (non-inferiority probability)
mask_post = x_range >= threshold
ax.fill_between(x_range[mask_post], post_pdf[mask_post], color="#d62728", alpha=0.3,
                label=f"Posterior tail area = {prob_non_inferior_post:.3f}")

# Optional: shade prior tail for comparison (lighter)
mask_prior = x_range >= threshold
ax.fill_between(x_range[mask_prior], prior_pdf[mask_prior], color="#1f77b4", alpha=0.15,
                label=f"Prior tail area = {prob_non_inferior_prior:.3f}")

# Decorations
ax.set_xlim(x_min, x_max)
ax.set_xlabel("Conversion rate p_A", fontsize=12)
ax.set_ylabel("Density", fontsize=12)
ax.set_title("Prior vs Posterior and Non-Inferiority Probability (tail area)", fontsize=14)
ax.grid(True, ls=":", alpha=0.5)
ax.legend(loc="best", fontsize=10)
plt.tight_layout()
plt.show()

print(f"Posterior P(p_A > {threshold:.2f}) = {prob_non_inferior_post:.4f} "
      f"({prob_non_inferior_post*100:.2f}%)")
print(f"Prior     P(p_A > {threshold:.2f}) = {prob_non_inferior_prior:.4f} "
      f"({prob_non_inferior_prior*100:.2f}%)")

## Picking the Best Variant

This is where the difference between **NHST** and a **Bayesian** approach becomes dramatic.  
Let’s compare the main options.

---

### 🧪 NHST Approaches


#### 1. Winner-Takes-All
- Pick the variant with the highest observed conversion rate.

**Problems:**  
- Ignores uncertainty and sampling noise.  
- Easily picks the wrong variant when samples are small.

#### 2. Pairwise *t*-Tests with Bonferroni Correction
- Run one test for every pair (A vs B, A vs C, B vs C).  
- Adjust the significance threshold to control false positives:  
  $$\alpha_\text{corrected} = \frac{0.05}{3} \approx 0.0167.$$

**Problems:**  
- Multiple comparisons inflate Type I error; Bonferroni is very conservative (higher Type II error).  
- Only gives “significant / not significant” — no direct probability of being best.

---

#### 3. ANOVA + Post-Hoc Tests
- One-way ANOVA checks if *any* difference exists, then post-hoc tests (Tukey, Dunnett, etc.) try to find which.

**Problems:**  
- Still needs multiple-comparison corrections.  
- ANOVA only says “something differs” — not which is best or by how much.

---

#### 4. Confidence Interval Overlap
- Compute 95% CIs for each variant and check for overlap.

**Problems:**  
- Overlapping CIs don’t mean “no difference.”  
- Often inconclusive and gives no probability a variant is best.

---

### 🌟 Bayesian Approach — Probability of Being Best

The Bayesian framework answers the question we actually care about:  
> *Which variant is most likely the best?*

**Method:**
1. Compute the **posterior Beta distribution** for each variant using its prior and observed data.  
2. Draw a large number of samples (e.g., 100k) from each posterior.  
3. For each simulated draw, identify which variant has the highest conversion rate.  
4. Report the probabilities:  
   $$P(A \text{ is best}),\; P(B \text{ is best}),\; P(C \text{ is best}), \ldots$$

---

**Advantages:**
-  **Direct answer:** “Variant B is best with 88.8% probability.”  
-  **Single coherent analysis:** no need for multiple-comparison corrections.  
-  **Scales naturally:** works the same way for 3, 5, 10, or 100 variants.  
-  **Quantifies uncertainty:** not just yes/no; can report $P(B>A)$, $P(B>A \;\&\; B>C)$, etc.  
-  **Flexible:** easily integrates prior knowledge and business context.  
-  **Business-friendly:** simple to factor in risk, cost, and implementation difficulty.  

---


**Key point:**  
Bayesian analysis gives a **probability each variant is best** — a direct, interpretable metric that scales cleanly and supports real-world decision making.


In [ ]:
# Three variants with similar conversion rates
# All are non-inferior (above 17% boundary) but we want the best one
variants = {
    'A': {'n': 800, 'x': 168},  # 21.0% conversion rate
    'B': {'n': 800, 'x': 172},  # 21.5% conversion rate
    'C': {'n': 800, 'x': 165}   # 20.625% conversion rate
}

print("\nVariant Data:")
print("-" * 80)
for name, data in variants.items():
    observed_rate = data['x'] / data['n']
    print(f"Variant {name}: n={data['n']:4d}, x={data['x']:3d}, "
          f"observed rate = {observed_rate:.4f} ({observed_rate*100:.2f}%)")

print(f"\nAll variants are above non-inferiority boundary: {control_group_conversion_rate - epsilon:.2f}")
print("But which one should we choose?")

All variants are above non-inferiority boundary: {control_group_conversion_rate - epsilon:.2f}
But which one should we choose?
In this specific case because all sample have the same size (n), we can just compare the mean but the "clean" way to do it is to run a Monte Carlo simulation to see which Variant would win "in all possible universes"

In [ ]:
# Compute posterior distributions (using non-informative prior Beta(1,1))
posteriors = {}

print("\nPosterior Distributions:")
print("-" * 80)

for name, data in variants.items():
    # Posterior parameters (with non-informative prior Beta(1,1))
    alpha_post = data['x'] + 1
    beta_post = data['n'] - data['x'] + 1
    
    # Posterior statistics
    posterior_mean = alpha_post / (alpha_post + beta_post)
    posterior_var = (alpha_post * beta_post) / \
                    ((alpha_post + beta_post)**2 * (alpha_post + beta_post + 1))
    posterior_std = np.sqrt(posterior_var)
    
    # Credible intervals
    ci_95_lower = beta_dist.ppf(0.025, alpha_post, beta_post)
    ci_95_upper = beta_dist.ppf(0.975, alpha_post, beta_post)
    
    posteriors[name] = {
        'alpha': alpha_post,
        'beta': beta_post,
        'mean': posterior_mean,
        'std': posterior_std,
        'ci_95': (ci_95_lower, ci_95_upper)
    }
    
    print(f"\nVariant {name}:")
    print(f"  Posterior: Beta(α={alpha_post}, β={beta_post})")
    print(f"  Posterior mean: {posterior_mean:.4f}")
    print(f"  Posterior std: {posterior_std:.4f}")
    print(f"  95% Credible Interval: [{ci_95_lower:.4f}, {ci_95_upper:.4f}]")

In [ ]:

# Create visualization of the three posterior distributions
fig, ax = plt.subplots(figsize=(12, 6))

colors = {'A': '#1f77b4', 'B': '#ff7f0e', 'C': '#2ca02c'}
x_range = np.linspace(0.15, 0.30, 1000)

for name in ['A', 'B', 'C']:
    alpha_p = posteriors[name]['alpha']
    beta_p = posteriors[name]['beta']
    mean_p = posteriors[name]['mean']
    ci_lower, ci_upper = posteriors[name]['ci_95']
    
    # Plot PDF
    pdf = beta_dist.pdf(x_range, alpha_p, beta_p)
    ax.plot(x_range, pdf, color=colors[name], lw=2.5, 
            label=f'{name}: mean={mean_p:.4f}')
    
    # Mark the mean
    ax.axvline(mean_p, color=colors[name], ls='--', lw=1, alpha=0.5)
    
    # Shade 95% credible interval
    mask = (x_range >= ci_lower) & (x_range <= ci_upper)
    ax.fill_between(x_range[mask], pdf[mask], alpha=0.2, color=colors[name])

# Add non-inferiority boundary
boundary = control_group_conversion_rate - epsilon
ax.axvline(boundary, color='red', ls=':', lw=2, 
           label=f'Non-inferiority boundary ({boundary:.2f})')

# Add control rate
ax.axvline(control_group_conversion_rate, color='black', ls=':', lw=2,
           label=f'Control rate ({control_group_conversion_rate:.2f})')

ax.set_xlabel('Conversion Rate', fontsize=12)
ax.set_ylabel('Posterior Density', fontsize=12)
ax.set_title('Posterior Distributions for Variants A, B, C', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, ls=':', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ All three posterior distributions overlap significantly")
print("  This shows there's uncertainty about which is truly best")

In [ ]:
# Run Monte Carlo simulation
n_simulations = 100000
print(f"Running {n_simulations:,} simulations...\n")

# Draw samples from each posterior
samples = {}
for name in ['A', 'B', 'C']:
    alpha_p = posteriors[name]['alpha']
    beta_p = posteriors[name]['beta']
    samples[name] = beta_dist.rvs(alpha_p, beta_p, size=n_simulations)

# For each simulation, determine which variant is best
best_counts = {'A': 0, 'B': 0, 'C': 0}

for i in range(n_simulations):
    # Get the sampled values for this simulation
    sample_values = {
        'A': samples['A'][i],
        'B': samples['B'][i],
        'C': samples['C'][i]
    }
    
    # Find which variant has the highest value in this simulation
    best_variant = max(sample_values, key=lambda k: sample_values[k])
    best_counts[best_variant] += 1

# Calculate probabilities
probabilities = {name: count / n_simulations for name, count in best_counts.items()}

print("RESULTS: Probability Each Variant is Best")
print("-" * 80)

for name in ['A', 'B', 'C']:
    prob = probabilities[name]
    bar = '█' * int(prob * 60)
    print(f"P({name} is best) = {prob:.4f} ({prob*100:5.2f}%) {bar}")

# Determine the winner
winner = max(probabilities, key=probabilities.get)
winner_prob = probabilities[winner]

print("\n" + "="*80)
print("BAYESIAN CONCLUSION:")
print("="*80)
print(f"✓ Variant {winner} is most likely the best")
print(f"  Probability: {winner_prob:.4f} ({winner_prob*100:.1f}%)")
print(f"\nInterpretation:")
print(f"  - There's a {winner_prob*100:.1f}% chance that {winner} has the highest true conversion rate")
print(f"  - This accounts for uncertainty in all three estimates")
print(f"  - Clear, actionable decision with quantified confidence")

### Clean Standalone function to compute non inferiorit and comparison

In [ ]:
import numpy as np
from scipy.stats import beta as beta_dist

def test_non_inferiority(n_control, x_control, variants_data, epsilon, 
                         alpha_prior=1, beta_prior=1, threshold=0.95):
    """
    Test non-inferiority of multiple variants against a control.
    
    Parameters:
    -----------
    n_control : int
        Number of samples in control group
    x_control : int
        Number of successes in control group
    variants_data : dict
        Dictionary with variant names as keys and {'n': samples, 'x': successes} as values
        Example: {'A': {'n': 1000, 'x': 200}, 'B': {'n': 1000, 'x': 215}}
    epsilon : float
        Non-inferiority margin (e.g., 0.03 for 3%)
    alpha_prior : float, optional
        Alpha parameter for Beta prior (default: 1 for uniform)
    beta_prior : float, optional
        Beta parameter for Beta prior (default: 1 for uniform)
    threshold : float, optional
        Probability threshold for declaring non-inferiority (default: 0.95)
    
    Returns:
    --------
    dict : Dictionary with results for each variant containing:
        - 'is_non_inferior': bool, whether variant is non-inferior
        - 'probability': float, P(variant > control - epsilon)
        - 'control_rate': float, posterior mean of control
        - 'variant_rate': float, posterior mean of variant
        - 'posterior_params': tuple, (alpha, beta) of variant posterior
    """
    # Control posterior
    alpha_control = x_control + alpha_prior
    beta_control = n_control - x_control + beta_prior
    control_rate = alpha_control / (alpha_control + beta_control)
    
    # Boundary for non-inferiority
    boundary = control_rate - epsilon
    
    results = {}
    n_simulations = 100000
    
    # Sample from control posterior once (reuse for all variants)
    control_samples = beta_dist.rvs(alpha_control, beta_control, size=n_simulations)
    
    for variant_name, data in variants_data.items():
        # Variant posterior
        alpha_variant = data['x'] + alpha_prior
        beta_variant = data['n'] - data['x'] + beta_prior
        variant_rate = alpha_variant / (alpha_variant + beta_variant)
        
        # Sample from variant posterior
        variant_samples = beta_dist.rvs(alpha_variant, beta_variant, size=n_simulations)
        
        # Compute P(variant > control - epsilon)
        prob_non_inferior = np.mean(variant_samples > (control_samples - epsilon))
        
        results[variant_name] = {
            'is_non_inferior': prob_non_inferior >= threshold,
            'probability': prob_non_inferior,
            'control_rate': control_rate,
            'variant_rate': variant_rate,
            'posterior_params': (alpha_variant, beta_variant)
        }
    
    return results


def select_best_variant(variants_data, alpha_prior=1, beta_prior=1, 
                       credible_level=0.95, n_simulations=100000):
    """
    Select the best variant among multiple options using Bayesian approach.
    
    Parameters:
    -----------
    variants_data : dict
        Dictionary with variant names as keys and {'n': samples, 'x': successes} as values
        Example: {'A': {'n': 800, 'x': 168}, 'B': {'n': 800, 'x': 172}}
    alpha_prior : float, optional
        Alpha parameter for Beta prior (default: 1 for uniform)
    beta_prior : float, optional
        Beta parameter for Beta prior (default: 1 for uniform)
    credible_level : float, optional
        Credible interval level (default: 0.95)
    n_simulations : int, optional
        Number of Monte Carlo simulations (default: 100000)
    
    Returns:
    --------
    dict : Dictionary containing:
        - 'best_variant': str, name of variant most likely to be best
        - 'probabilities': dict, P(each variant is best)
        - 'posterior_means': dict, posterior mean for each variant
        - 'credible_intervals': dict, (lower, upper) credible interval for each variant
        - 'expected_loss': dict, expected loss from choosing each variant
    """
    variant_names = list(variants_data.keys())
    posteriors = {}
    samples = {}
    
    # Compute posteriors and draw samples
    for name, data in variants_data.items():
        alpha_post = data['x'] + alpha_prior
        beta_post = data['n'] - data['x'] + beta_prior
        
        posteriors[name] = {
            'alpha': alpha_post,
            'beta': beta_post,
            'mean': alpha_post / (alpha_post + beta_post)
        }
        
        # Draw samples from posterior
        samples[name] = beta_dist.rvs(alpha_post, beta_post, size=n_simulations)
        
        # Compute credible interval
        ci_lower = beta_dist.ppf((1 - credible_level) / 2, alpha_post, beta_post)
        ci_upper = beta_dist.ppf(1 - (1 - credible_level) / 2, alpha_post, beta_post)
        posteriors[name]['credible_interval'] = (ci_lower, ci_upper)
    
    # Monte Carlo: count how often each variant is best
    best_counts = {name: 0 for name in variant_names}
    
    for i in range(n_simulations):
        # Get samples for this iteration
        sample_values = {name: samples[name][i] for name in variant_names}
        
        # Find best variant in this simulation
        best_variant = max(sample_values, key=sample_values.get)
        best_counts[best_variant] += 1
    
    # Calculate probabilities
    probabilities = {name: count / n_simulations for name, count in best_counts.items()}
    
    # Expected loss: E[max(all) - this variant]
    expected_loss = {}
    for name in variant_names:
        max_samples = np.maximum.reduce([samples[v] for v in variant_names])
        losses = max_samples - samples[name]
        expected_loss[name] = np.mean(losses)
    
    # Determine best variant
    best_variant = max(probabilities, key=probabilities.get)
    
    return {
        'best_variant': best_variant,
        'probabilities': probabilities,
        'posterior_means': {name: posteriors[name]['mean'] for name in variant_names},
        'credible_intervals': {name: posteriors[name]['credible_interval'] 
                              for name in variant_names},
        'expected_loss': expected_loss
    }


# Example usage:
if __name__ == "__main__":
    # Non-inferiority test
    print("="*80)
    print("NON-INFERIORITY TEST")
    print("="*80)
    
    n_control = 7000
    x_control = 1400
    variants = {
        'A': {'n': 1000, 'x': 200},
        'B': {'n': 1000, 'x': 215},
        'C': {'n': 1000, 'x': 195}
    }
    
    non_inf_results = test_non_inferiority(
        n_control=n_control,
        x_control=x_control,
        variants_data=variants,
        epsilon=0.03,
        threshold=0.95
    )
    
    for variant, result in non_inf_results.items():
        print(f"\nVariant {variant}:")
        print(f"  Control rate: {result['control_rate']:.4f}")
        print(f"  Variant rate: {result['variant_rate']:.4f}")
        print(f"  P(variant > control - 0.03): {result['probability']:.4f}")
        print(f"  Non-inferior? {'YES ✓' if result['is_non_inferior'] else 'NO ✗'}")
    
    # Select best variant (only among non-inferior ones)
    print("\n" + "="*80)
    print("BEST VARIANT SELECTION")
    print("="*80)
    
    # Filter to only non-inferior variants
    non_inferior_variants = {
        name: data for name, data in variants.items()
        if non_inf_results[name]['is_non_inferior']
    }
    
    if non_inferior_variants:
        best_results = select_best_variant(non_inferior_variants)
        
        print(f"\nBest variant: {best_results['best_variant']}")
        print(f"\nProbabilities of being best:")
        for name, prob in best_results['probabilities'].items():
            print(f"  {name}: {prob:.4f} ({prob*100:.1f}%)")
        
        print(f"\nPosterior means:")
        for name, mean in best_results['posterior_means'].items():
            print(f"  {name}: {mean:.4f}")
        
        print(f"\n95% Credible Intervals:")
        for name, (lower, upper) in best_results['credible_intervals'].items():
            print(f"  {name}: [{lower:.4f}, {upper:.4f}]")
        
        print(f"\nExpected loss (if you choose this variant):")
        for name, loss in best_results['expected_loss'].items():
            print(f"  {name}: {loss:.6f} ({loss*100:.4f}%)")
    else:
        print("\n⚠️  No variants are non-inferior to control!")

In [ ]:
def nhst_non_inferiority_test(n_control, x_control, n_variant, x_variant, 
                                epsilon, alpha=0.05, h1_effect_size=0.0):
    """
    Perform NHST non-inferiority test for a single variant against control.
    
    Parameters:
    -----------
    n_control : int
        Number of samples in control group
    x_control : int
        Number of successes in control group
    n_variant : int
        Number of samples in variant group
    x_variant : int
        Number of successes in variant group
    epsilon : float
        Non-inferiority margin (e.g., 0.03 for 3% acceptable degradation)
    alpha : float, optional
        Significance level for Type I error (default: 0.05)
    h1_effect_size : float, optional
        Expected effect size under H1 for power calculation (default: 0.0, meaning no difference)
    
    Returns:
    --------
    dict : Dictionary containing:
        - 'decision': str, 'REJECT H0' or 'FAIL TO REJECT H0'
        - 'p_value': float, probability of observed result under H0
        - 'power': float, probability of rejecting H0 when H1 is true
        - 'observed_difference': float, observed difference in proportions (variant - control)
        - 'critical_value': float, threshold for rejection at significance level alpha
        - 'se_h0': float, standard error under H0 (boundary case)
        - 'se_h1': float, standard error under H1
        - 'control_rate': float, observed control conversion rate
        - 'variant_rate': float, observed variant conversion rate
        - 'z_statistic': float, standardized test statistic
    """
    from scipy.stats import norm
    
    # Observed proportions
    p_control = x_control / n_control
    p_variant = x_variant / n_variant
    observed_diff = p_variant - p_control
    
    # Standard error under H0 (unpooled, since we're not assuming p_A = p_C)
    se_h0 = np.sqrt((p_control * (1 - p_control) / n_control) + 
                     (p_variant * (1 - p_variant) / n_variant))
    
    # Hypothesis test under boundary H0: E[Delta] = -epsilon
    mu_h0 = -epsilon
    
    # Compute p-value (right-tail test: H1 is Delta > -epsilon)
    p_value = norm.sf(observed_diff, loc=mu_h0, scale=se_h0)
    
    # Critical value for rejection at significance level alpha
    critical_value = norm.isf(alpha, loc=mu_h0, scale=se_h0)
    
    # Decision
    decision = 'REJECT H0' if p_value <= alpha else 'FAIL TO REJECT H0'
    
    # Z-statistic (standardized)
    z_statistic = (observed_diff - mu_h0) / se_h0
    
    # Power calculation under H1
    # Under H1, assume true effect is h1_effect_size (e.g., 0 for "no worse")
    # Use pooled SE if h1_effect_size = 0 (assuming p_variant = p_control under H1)
    if h1_effect_size == 0.0:
        # Pooled proportion for H1
        p_pooled = (x_control + x_variant) / (n_control + n_variant)
        se_h1 = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_control + 1/n_variant))
    else:
        # Use unpooled SE with assumed rates under H1
        # Assume variant has control_rate + h1_effect_size
        p_variant_h1 = p_control + h1_effect_size
        se_h1 = np.sqrt((p_control * (1 - p_control) / n_control) + 
                        (p_variant_h1 * (1 - p_variant_h1) / n_variant))
    
    # Under H1, the distribution is centered at h1_effect_size
    mu_h1 = h1_effect_size
    
    # Power = P(reject H0 | H1 is true) = P(observed_diff > critical_value | H1)
    # This is the right-tail probability of the H1 distribution beyond critical_value
    power = norm.sf(critical_value, loc=mu_h1, scale=se_h1)
    
    return {
        'decision': decision,
        'p_value': p_value,
        'power': power,
        'observed_difference': observed_diff,
        'critical_value': critical_value,
        'se_h0': se_h0,
        'se_h1': se_h1,
        'control_rate': p_control,
        'variant_rate': p_variant,
        'z_statistic': z_statistic
    }


# Example usage with the notebook's data
if __name__ == "__main__":
    print("="*80)
    print("NHST NON-INFERIORITY TEST")
    print("="*80)
    
    # Use the same data from the notebook
    result = nhst_non_inferiority_test(
        n_control=7000,
        x_control=1400,  # 20% rate
        n_variant=150,
        x_variant=33,
        epsilon=0.03,
        alpha=0.05,
        h1_effect_size=0.0
    )
    
    print(f"\nObserved Data:")
    print(f"  Control: n={7000}, x={1400}, rate={result['control_rate']:.4f}")
    print(f"  Variant: n={150}, x={33}, rate={result['variant_rate']:.4f}")
    print(f"  Observed difference (variant - control): {result['observed_difference']:.4f}")
    
    print(f"\nTest Configuration:")
    print(f"  Non-inferiority margin (ε): {0.03:.3f}")
    print(f"  Significance level (α): {0.05:.3f}")
    print(f"  H1 effect size: {0.0:.3f}")
    
    print(f"\nTest Results:")
    print(f"  Decision: {result['decision']}")
    print(f"  p-value: {result['p_value']:.4f}")
    print(f"  z-statistic: {result['z_statistic']:.4f}")
    print(f"  Critical value: {result['critical_value']:.4f}")
    
    print(f"\nStandard Errors:")
    print(f"  SE under H0 (boundary): {result['se_h0']:.4f}")
    print(f"  SE under H1 (pooled): {result['se_h1']:.4f}")
    
    print(f"\nPower Analysis:")
    print(f"  Power (1 - β): {result['power']:.4f} ({result['power']*100:.1f}%)")
    print(f"  Type II error (β): {1 - result['power']:.4f} ({(1-result['power'])*100:.1f}%)")
    
    print(f"\nInterpretation:")
    if result['decision'] == 'REJECT H0':
        print(f"  ✓ The variant is statistically non-inferior to control at α={0.05}")
        print(f"    (p-value {result['p_value']:.4f} < {0.05})")
    else:
        print(f"  ✗ Cannot conclude non-inferiority at α={0.05}")
        print(f"    (p-value {result['p_value']:.4f} > {0.05})")
    
    print(f"\n  Power is {result['power']*100:.1f}% - this test has a {(1-result['power'])*100:.1f}% chance")
    print(f"  of missing non-inferiority even if variant truly has no degradation.")
    
    if result['power'] < 0.8:
        print(f"\n  ⚠️  WARNING: Power is below conventional threshold of 80%!")
        print(f"     Consider increasing sample size for more reliable results.")